# Loss Functions

In this exercise, you will compare the effects of Loss functions on a `LinearRegression` model.

👇 Let's download a CSV file to use for this challenge and parse it into a DataFrame

In [ ]:
import pandas as pd

data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/loss_functions_dataset.csv")
data.sample(5)

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area,Average Temperature
711,0.66,759.5,318.5,220.50,3.5,0.4,16.770
230,0.66,759.5,318.5,220.50,3.5,0.1,13.165
74,0.74,686.0,245.0,220.50,3.5,0.1,11.920
172,0.71,710.5,269.5,220.50,3.5,0.1,12.455
721,0.98,514.5,294.0,110.25,7.0,0.4,32.975


🎯 Your task is to predict the average temperature inside a greenhouse based on its design. Your temperature predictions will help you select the appropriate greenhouse design for each plant, based on their climate needs. 

🌿 You know that plants can handle small temperature variations, but are exponentially more sensitive as the temperature variations increase. 

## 1. Theory 

❓ Theoretically, which Loss function would you train your model on to limit the risk of killing plants?

<details>
<summary> 🆘 Answer </summary>
    
By theory, you would use a Mean Square Error (MSE) Loss function. It would penalize outlier predictions and prevent your model from committing large errors. This would ensure smaller temperature variations and a lower risk for plants.

</details>

## 2. Application

### 2.1 Preprocessing

❓ Standardise the features

In [ ]:
from sklearn.preprocessing import StandardScaler

# Select only the features 
X = data.loc[:,'Relative Compactness':'Glazing Area']

# Fit scaler
scaler = StandardScaler().fit(X)

# Scale continuous features 
X_scaled = scaler.transform(X)

### 2.2 Modeling

In this section, you are going to verify the theory by evaluating models optimized on different Loss functions.

### Least Squares (MSE) Loss

❓ **10-Fold Cross-validate** a Linear Regression model optimized by **Stochastic Gradient Descent** (SGD) on a **Least Squares Loss** (MSE)



In [ ]:
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.linear_model import SGDRegressor

# Squared loss SGD Regressor
sgd_model = SGDRegressor(loss="squared_error")

# Cross Validate Model
sgd_model_cv = cross_validate(
    sgd_model, 
    X_scaled, 
    data['Average Temperature'],
    cv = 10, 
    scoring = ['r2','max_error']
)

sgd_model_cv

{'fit_time': array([0.00405931, 0.00180626, 0.00203586, 0.00227308, 0.00195575,
        0.00231194, 0.00253916, 0.00243878, 0.00203109, 0.00214481]),
 'score_time': array([0.00038481, 0.00017571, 0.00016403, 0.00017786, 0.00027323,
        0.00018096, 0.00028801, 0.00025821, 0.00017381, 0.00016212]),
 'test_r2': array([0.78760228, 0.90600811, 0.89521719, 0.88375495, 0.93107145,
        0.89643488, 0.92656466, 0.91619262, 0.89598907, 0.93967098]),
 'test_max_error': array([-9.87443679, -8.64316656, -8.80351335, -9.14971253, -8.96393795,
        -8.60473625, -8.54151666, -8.79112051, -8.39854722, -7.64867915])}

❓ Compute 
- the mean cross-validated R2 score and save it in the variable `r2`
- the single biggest prediction error in °C of all your folds and save it in the variable `max_error_celsius`?

(Tips: `max_error` is an accepted scoring metric in sklearn)

In [ ]:
r2 = sgd_model_cv['test_r2'].mean()
r2

0.8978506177363028

In [ ]:
max_error_celsius = abs(sgd_model_cv['test_max_error']).max()
max_error_celsius

9.874436792526318

### Mean Absolute Error (MAE) Loss

What if we optimize our model on the MAE instead?

❓ **10-Fold Cross-validate** a Linear Regression model optimized by **Stochastic Gradient Descent** (SGD) on a **MAE** Loss

<details>
<summary>💡 Hints</summary>

- MAE loss cannot be directly specified in `SGDRegressor`. It must be engineered by adjusting the right parameters

</details>

In [ ]:
# MAE loss engineered by setting epsilon_insensitive = 0
mae_model = SGDRegressor(loss="epsilon_insensitive", epsilon = 0)

# Cross Validate Model
mae_sgd = cross_validate(
    mae_model, 
    X_scaled, 
    data['Average Temperature'], 
    cv = 10,  
    scoring = ['r2','max_error']
)

❓ Compute 
- the mean cross-validated R2 score, store it in `r2_mae`
- the single biggest prediction error of all your folds, store it in `max_error_mae`?

In [ ]:
r2_mae = mae_sgd['test_r2'].mean()
r2_mae

0.8760129203893943

In [ ]:
max_error_mae = abs(mae_sgd['test_max_error']).max()
max_error_mae

11.182139855714794

## 3. Conclusion

❓Which of the models you evaluated seems the most appropriate for your task?

<details>
<summary> 🆘Answer </summary>
    
Although mean cross-validated r2 scores are approximately similar between the two models, the one optimized on a MAE has more chance to make larger mistakes from time to time, increasing the risk of killing plants!

    
</details>

# 🏁 Check your code and push your notebook

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'loss_functions',
    r2 = r2,
    r2_mae = r2_mae,
    max_error = max_error_celsius,
    max_error_mae = max_error_mae
)

result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/olivergiles/.pyenv/versions/3.10.6/envs/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/olivergiles/code/lewagon_dev/data-solutions/05-ML/04-Under-the-hood/01-Loss-Functions/tests
plugins: asyncio-0.19.0, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 3 items

test_loss_functions.py::TestLossFunctions::test_max_error_order PASSED   [ 33%]
test_loss_functions.py::TestLossFunctions::test_r2 PASSED                [ 66%]
test_loss_functions.py::TestLossFunctions::test_r2_mae PASSED            [100%]

============================== 3 passed in 0.04s ===============================


💯 You can commit your code:

git add tests/loss_functions.pickle

git commit -m 'Completed loss_functions step'

git push origin master

